# Introduction
In this notebook, we implement a simple Tic Tac Toe game, then write a guidance program to make a language model play Tic Tac Toe

In [4]:
import guidance
from guidance import models, gen, any_char, any_char_but, regex, substring, substring_no_empty, with_temperature

In [2]:
mistral = models.LlamaCpp("/Users/nicholasking/code/models/mixtral-8x7b-instruct-v0.1.Q3_K_M.gguf", n_gpu_layers=-1, n_ctx=4096)

In [3]:
class TicTacToe:
    def __init__(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.current_player = 'X'

    def is_valid_move(self, x, y):
        return 0 <= x < 3 and 0 <= y < 3 and self.board[x][y] == ' '

    def make_move(self, x, y):
        if self.is_valid_move(x, y):
            self.board[x][y] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
            return True
        return False

    def check_winner(self):
        # Check rows, columns and diagonals for a winner
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != ' ':
                return self.board[i][0]
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != ' ':
                return self.board[0][i]
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != ' ':
            return self.board[0][0]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != ' ':
            return self.board[0][2]
        return None

    def is_draw(self):
        return all(self.board[x][y] != ' ' for x in range(3) for y in range(3))

    def __str__(self):
        rows = [' | '.join(row) for row in self.board]
        return '\n---------\n'.join(rows)

In [21]:
@guidance
def tic_tac_toe_game(lm, game):
    lm += f"Current board:\n{game}\n"
    if game.check_winner() or game.is_draw():
        lm += f"Game over! Winner: {game.check_winner() if game.check_winner() else 'No one'}\n"
        return lm
    
    lm += f"Player {game.current_player}, choose your move (row, column): "
    possible_moves = [(x, y) for x in range(3) for y in range(3) if game.is_valid_move(x, y)]
    possible_move_regex = "|".join([f"({x}, {y})" for x, y in possible_moves])
    lm += gen(name="move", stop="\n", regex=possible_move_regex, temperature=0.8)
    x, y = eval(lm["move"])
    game.make_move(x, y)
    return lm

In [23]:
game = TicTacToe()
while not game.check_winner() and not game.is_draw():
    lm = mistral + tic_tac_toe_game(game)

print("Final board:")
print(game)

Final board:
O |   |  
---------
X | X | X
---------
  | O |  
